In [2]:
# This program aims to generate Data Sequences
# A A Data Sequence consists of the following parts:
# 1. A PCR primer pair which can also be used as Chapter Number
# 2. A 4-nt base-3 converted sequence which denotes the Segment Number
# 3. Series of 7-nt Data Units which represents a 16-bit binary number. The first 6-nt refers to the Group Index, the last "Pointer Base" can be used to map the Reference Pool.
# 4. 12-nt base-3 converted error correction code which can correct up to 1 substitution error in the original Data Sequence.


import math
from array import *
import collections
from reedsolo import RSCodec

baseNumber = 0
baseNumber_complete = 0

# 2 error correction codes
rsc = RSCodec(2)

%store -r my_hexdata
%store -r Groups

bitsNumber = len(my_hexdata)*4

# Define Base Mapping Matrix
baseMappingMatrix = ['ACGT', 'CGTA', 'GTAC', 'TACG']

%store -r primerLibrary
%store -r segmentedChapteredData

def DNA_reverse_complement(DNA):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join(complement.get(base, base) for base in reversed(DNA))

def getPosition(String):
    position_list = []
    for i in range (0, 16):
        if String[i] == '1':
            position_list.append(i)
    return position_list

def converter(seq):
    converter = {'A': '00', 'C': '01', 'G': '10', 'T': '11'} 
    bases = list(seq) 
    bases = [converter[base] for base in bases] 
    return ''.join(bases)

def deconverter(seq):
    deconverter = {'00': 'A', '01': 'C', '10': 'G', '11': 'T'} 
    doubleBits = [seq[i:i+2] for i in range(0, len(seq), 2)]
    doubleBits = [deconverter[doubleBit] for doubleBit in doubleBits] 
    return ''.join(doubleBits)

def GCPercent(seq):
    counts = collections.Counter(seq)
    percent = (counts['C'] + counts['G'])/float(len(seq)) 
    return percent

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))
    
def base3ToOligo(seq):
    base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
    base3ToOligoConverted = []
    for i in range (0, len(seq)):
        base3ToOligoConverted.append(base3ToOligo[seq[i]])
        if base3ToOligo[seq[i]] == 'C':
            base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
        elif base3ToOligo[seq[i]] == 'G':
            base3ToOligo = {'0': 'T', '1': 'A', '2': 'C'}
        elif base3ToOligo[seq[i]] == 'T':
            base3ToOligo = {'0': 'A', '1': 'C', '2': 'G'}
        elif base3ToOligo[seq[i]] == 'A':
            base3ToOligo = {'0': 'C', '1': 'G', '2': 'T'}
    return ''.join(base3ToOligoConverted)

def molmass(DNA):
    weight = 0
    single_mass = [313.2, 304.2, 289.2, 329.2]   # Corresponding to base "A", "T", "C", "G", respectively
    for i in range (0, len(DNA)):
        if DNA[i] == 'A':
            weight += single_mass[0]
        elif DNA[i] == 'T':
            weight += single_mass[1]
        elif DNA[i] == 'C':
            weight += single_mass[2]
        elif DNA[i] == 'G':
            weight += single_mass[3]
    return weight

def molconc(mass_conc, mol_weight):   # The unit of mass_conc is 'ng/uL', the unit of mol_weight is 'g/mol', the unit of the result mol_conc is 'nM'
    mol_conc = (mass_conc * 10**6) / (mol_weight * 10**9) *10**9
    return mol_conc
    
    
chapteredDataBinaryTotal = []    # Storing the chaptered binary data

scale = 16 ## equals to hexadecimal
mass = 0  # Oligo pool yield
GCTotal = 0  # Storing the sum of GC content of every data strand
count = 0  # Total number of data strands
chapteredDataBinary = []   # Storing the every unit of chaptered binary data

# Initialization 
array_full = [[None]*100 for _ in range(40)]   # Storing the complete data sequence in 2D-matrix with notation of chapter and segment index
array_data = [[None]*100 for _ in range(40)]   # Storing actual data sequence + RS-correction bases in 2D-matrix with notation of chapter and segment index
array_data_payload = [[None]*100 for _ in range(40)]   # Storing actual data sequence in 2D-matrix with notation of chapter and segment index

# Preparation of segmented data whose length should be a multiple of 4 to facilitate RS correction
for i in range (0, len(segmentedChapteredData)):
    for k in range (0, len(segmentedChapteredData[i])):
        a = bin(int(segmentedChapteredData[i][k], scale))[2:].zfill(8)
        if (len(a) % 4 != 0):
            a = '0' * (4-(len(a)%4)) + a
        chapteredDataBinary.append(a)
    chapteredDataBinaryTotal.append(chapteredDataBinary)
    chapteredDataBinary = []
    
# Assembling Data Sequences
for Chapter in range (0, len(chapteredDataBinaryTotal)):
    for Segment in range (0, len(chapteredDataBinaryTotal[Chapter])):
        originalData = chapteredDataBinaryTotal[Chapter][Segment]
        originalDataList = []
        dataPositions = []
        dataPositionsUnits = []
        arrayGap = 16    # Grouping interval
        arraySegmentNumber = int(len(originalData)/arrayGap)
        for j in range (0, arraySegmentNumber):
            originalDataSegment = originalData[arrayGap*j:arrayGap*(j+1)]
            originalDataList.append(originalDataSegment)
            dataPositionsUnits = getPosition(originalDataSegment)   
            dataPositions.append(dataPositionsUnits)
            dataPositionsUnits = []

        firstIndex = []  # Group index
        dataSequence = []   # Storing the original data sequence
        dataSequence_tuned = []  # Storing the RS-corrected data sequence
        for i in range (0, len(dataPositions)):
            for j0 in range (0, len(Groups)):
                for j1 in range (0, len(Groups[j0])):
                    if dataPositions[i] == Groups[j0][j1]:
                        firstIndex.append(j0)
                        ternaryNumber = ternary(j0)
                        ternaryNumber = '0'*(6-len(ternaryNumber)) + ternaryNumber
                        dataSequence.append(base3ToOligo(ternaryNumber))
                        dataSequence_tuned.append(base3ToOligo(ternaryNumber))
                        dataSequence.append(baseMappingMatrix[j1][j0%4])
                        dataSequence_tuned.append(baseMappingMatrix[j1][j0%4])   # Create dataSequence for RS correction
                        dataSequence_tuned.append(baseMappingMatrix[j1][j0%4])   # Supplement one base to make the length of the converted binary number exactly 16, thereby convenient for RS correction

        dataSequence = ''.join(dataSequence)  
        array_data_payload[Chapter][Segment] = dataSequence
        dataSequence_tuned = ''.join(dataSequence_tuned)
        percent = GCPercent(dataSequence)
        ternarySegmentNumber = ternary(Segment)
        
        # Generate segment index sequence
        ternarySegmentNumber = '0'*(4-len(ternarySegmentNumber)) + ternarySegmentNumber
        ternarySegmentNumberSeq = base3ToOligo(ternarySegmentNumber)
        # Actual data appended with segment index sequences
        dataSequence = ternarySegmentNumberSeq + dataSequence
        dataSequence_tuned = ternarySegmentNumberSeq + dataSequence_tuned
        # Generate RS correcting codes
        binaryConverted = converter(dataSequence_tuned)
        binaryList = [int(binaryConverted[i:i + 8], 2) for i in range(0, len(binaryConverted), 8)]
        byteArray = rsc.encode(binaryList)
        byte_to_ternary_oligo = []
        for byte in byteArray[-2:]:
            byte_to_ternary = ternary(byte)
            byte_to_ternary = '0'*(6-len(byte_to_ternary)) + byte_to_ternary
            byte_to_ternary_oligoUnit = base3ToOligo(byte_to_ternary)
            byte_to_ternary_oligo.append(byte_to_ternary_oligoUnit)
        byte_to_ternary_oligo = ''.join(byte_to_ternary_oligo)
        # Concatenation to build a complete data sequence
        baseDeconverted = primerLibrary[Chapter] + dataSequence + byte_to_ternary_oligo + primerLibrary[len(primerLibrary)-Chapter-1]
        array_data[Chapter][Segment] = dataSequence + byte_to_ternary_oligo
        array_full[Chapter][Segment] = baseDeconverted
        print('This is Chapter '+ str(Chapter))
        print('This is Segment '+ str(Segment))
        count += 1
        print(baseDeconverted)
        # Calculating GC content of each strand
        GC = GCPercent(baseDeconverted)
        print('GC content of this strand: ' + str(GC))
        GCTotal += GCPercent(baseDeconverted)
        mass += molmass(baseDeconverted)
        percent = GCPercent(baseDeconverted)
        print("Oligo Length: " + str(len(baseDeconverted)))
        baseNumber += len(dataSequence)
        baseNumber_complete += len(baseDeconverted)
        

ref_base = 80*9*16      # Payload eference base number excluding the primers, indexes and RS correcting bases       
ref_base_complete = 142*9*16    # Complete reference base number
totalBaseNumber = baseNumber + ref_base  # Sum of data and reference payload base number 
totalBaseNumber_complete = baseNumber_complete + ref_base_complete   # Sum of data and reference xomplete base number

print("Number of bits: " + str(bitsNumber))
file_size = bitsNumber/8/1000
print('File size: ' + str(file_size) + 'KB') 
print('Number of Bases in the payload of Data Pool: ' + str(baseNumber))
print('Number of Bases in the payload of Reference Pool: ' + str(ref_base))
print('Number of total payload bases: ' + str(totalBaseNumber))
print('Number of Bases in complete Data Pool: ' + str(baseNumber_complete))
print('Number of Bases in complete Reference Pool: ' + str(ref_base_complete))
print('Number of total bases: ' + str(totalBaseNumber_complete))

# Calculation of coding potential
codingPotential_payload = bitsNumber/totalBaseNumber
print('Payload coding potential: ' + str(codingPotential_payload))
codingPotential_complete = bitsNumber/totalBaseNumber_complete
print('Overall coding potential: ' + str(codingPotential_complete))

# Calculating average GC content
GCAverage = GCTotal/count
print('Average GC content: ' + str(GCAverage))
# Calculation of concentration of the dissolved oligo pool
print("Total molecular weight: " + str(mass) + ' g/mol')
mass_concentration = 0.434
molecular_concentration = molconc(mass_concentration, mass)
print("Oligo pool concentration: " + str(molecular_concentration) + ' nM')

%store array_full
%store array_data
%store array_data_payload

print(count)    # Total number of data strands 


This is Chapter 0
This is Segment 0
ACTCCTCGAATGCTAGCGTTGGTACGTACGTAGTACGACGTACGCGGTACTATGTACTCAGTACTGCGTACACGGTACAGTGTACATAATGAGCAAGCACTAAGCACTAGCTCTGGTCATAACGGTAGCTTCCTGTATGCCT
GC content of this strand: 0.4859154929577465
Oligo Length: 142
This is Chapter 0
This is Segment 1
ACTCCTCGAATGCTAGCGTTGGTAGGTAGTACGTAGTCGGTAGTGTGTAGACAGTAGAGCGTACATAGTAGATGGTAGCGTGTAGCTAGTAGCACGTATACGGTATAGTGTATATAGTATCGCGTATCTGGCTAGAGTGCTCACGGTAGCTTCCTGTATGCCT
GC content of this strand: 0.4723926380368098
Oligo Length: 163
This is Chapter 0
This is Segment 2
ACTCCTCGAATGCTAGCGTTGGTATGTATCATGTATGTAGTATGACGTACAGTGTACATAGTATGCGGTCGTATGTATATAGTCGTCAGTCGTGCGTATCTGGTCGACGGTCGAGTGTCGATAAGCACTAGTCGTGGCGAGCACGGTAGCTTCCTGTATGCCT
GC content of this strand: 0.48466257668711654
Oligo Length: 163
This is Chapter 0
This is Segment 3
ACTCCTCGAATGCTAGCGTTGGTCGGTAGTCGGTCGCGCGTCGCTGGTATCTGGTCGCATGTCTACAGTCTAGCGTCTATGGTCTCGTGTCTCTAGTCTCACGTCTGTGGTCTGATGTCTGCAAGCACTAGCGTCTGTGAGCACGGTAGCTTCCTGTATGCCT
GC content of this strand: 0

This is Chapter 0
This is Segment 52
ACTCCTCGAATGCTAGCGTTGTGCTGTCAGACGTCAGCGGTCATATGTCGCATGTCATCAGATGCGTGTGACTAGTCGCATGTAGATGGTGCTCAGCTATATGTCAGTAGCTCGTGGACGTCGTAGAGACGGTAGCTTCCTGTATGCCT
GC content of this strand: 0.5167785234899329
Oligo Length: 149
This is Chapter 1
This is Segment 0
ACATCGCCATTCCAAGAGCATGTACGCTCGATGTCTCGTGCTCGCAGCTACATGCTATATGCTCTACGTAGCACGCGATCGATACAGTATACACTGCTCTCGAGCACTAGTGTACGTAGATTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.49295774647887325
Oligo Length: 142
This is Chapter 1
This is Segment 1
ACATCGCCATTCCAAGAGCATGTAGGTACTCAGCTATATGTAGACAGCTCTGTGACTGCGGCTCACAGCTCAGCGAGCTGTGTGCGTAGATCTATGCTCATGGTATCGCGTCTCGTAGCAGATGTCTGTGAGATGTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5064102564102564
Oligo Length: 156
This is Chapter 1
This is Segment 2
ACATCGCCATTCCAAGAGCATGTATGCTGTATGACGTCGGTAGATGGTGAGCAGTCATATGTCTATGGTCGTATGATCTATGACTGCGGTGCGACGTACAGTGTCTGCAAGCACTAGCGCACGCGTCGTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5100671140939598
Oligo Length: 14

This is Chapter 1
This is Segment 34
ACATCGCCATTCCAAGAGCATTATCGCAGATGGCGACGTGTGTCTGGTACTGCTACTAGCGTCTATGGTCTCGTGTCTCTAGTAGTCGGTCGCGCGTATCATGTCTGCAGTCACGCGTGTGACGTCTGATGTCTGTGGTCGTCGCTGTGTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5352941176470588
Oligo Length: 170
This is Chapter 1
This is Segment 35
ACATCGCCATTCCAAGAGCATTATGGCTGCTGGCTGCATGACTAGTGACACTAGACGAGCGACGCGTGTCTGTGGTGACTAGTATCTGAGCACTAAGCACTAAGCACTAGCTGCGGACTGTTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5140845070422535
Oligo Length: 142
This is Chapter 1
This is Segment 36
ACATCGCCATTCCAAGAGCATTCGTTACTCGTGTCTATGGTCTCGTGTGCGTAGATCACGGACGCGTGTCACGCGCGCGCGGTAGTCGGCTACGCGACTACGGCTGTCAGACGATGGATACGTGAGTGTGCATATTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5512820512820513
Oligo Length: 156
This is Chapter 1
This is Segment 37
ACATCGCCATTCCAAGAGCATTCGAGCAGATGGCGACGTGTACATAGTCACGCATAGACAATACTCCGTCGACGGTACAGTGCTACTGGTACAGTGTACATAAGCACACGAGTGAGCTCGCTCAACTGGTGATTCGTGCAAC
GC content of this strand: 0.5070422535211268


This is Chapter 2
This is Segment 32
AGATTGCACGTGTGTCTGCCTTAGCGATCGACGACACTAGACTACGGTACTATGCATAGTGTGTCATGTAGTCGGCGTAGTATGAGCAGTAGCACGTATATAGCTCGCAGTATCTGGCACGACGTACGCGAGCACTAGAGATGGTGCTCACAGACACGTTCGATACCGCA
GC content of this strand: 0.5117647058823529
Oligo Length: 170
This is Chapter 2
This is Segment 33
AGATTGCACGTGTGTCTGCCTTATAGTCGCATGAGTGATGTCTGCATACACATGCACGACGACTGCGTAGTATATAGTCGCGTGAGCAGACTAGTAGCACTAAGCACTAGTGTATGAGAGAACAGACACGTTCGATACCGCA
GC content of this strand: 0.4788732394366197
Oligo Length: 142
This is Chapter 2
This is Segment 34
AGATTGCACGTGTGTCTGCCTTATCTAGCTCAGTGCGTAGCTGACGGAGAGACTAGCTGCGAGTCACTAGCACGATAGCGTGTAGTCGGTCGCGCGTAGCACAGCACTATACTACGCACTCACAGACACGTTCGATACCGCA
GC content of this strand: 0.5422535211267606
Oligo Length: 142
This is Chapter 2
This is Segment 35
AGATTGCACGTGTGTCTGCCTTATGGTGATGTGTGATCGGCGCATAGTGAGCAGCGACGTGTATCTGTAGTATATAGTCGCGTGAGCAGACTAGTGACACGTGAGACATTAGACACGCATCTGGTGTAGTAGCACTATACTATGCAGCGACAGACACGTTCGATACCGCA
GC content of this strand: 0.494

This is Chapter 4
This is Segment 7
AGATGGCATGGACACGGAGATGTGAGTCGACGGCTCGCAGACATACGTATCATTCGACATGATCACGGATCGCGGCGTCTGGACTACGGTACAGTGTACATAGTCGTGCGTGCACGGCAGTGTGACGTAGTAGACTCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.5448717948717948
Oligo Length: 156
This is Chapter 4
This is Segment 8
AGATGGCATGGACACGGAGATGTGCATGAGCCATACGCTAGCACAGTCGAGTAGTGTGCGGAGTGATTCGAGACTCGAGCGTCGTCGTTCGTGCAGACTGCGGTCTACAAGCAGATGCATACGAGCTCTCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.5503355704697986
Oligo Length: 149
This is Chapter 4
This is Segment 9
AGATGGCATGGACACGGAGATGACGTACTCGTGCTATATGCACTGCGATAGATGCTACATGCTATATTCGTGCAGCATCTGGTACAGTGTACATAGACGATGGATACGTGACTGTAGTAGAGCGTATCATGTCGCGCGCTGCAGTCAGTTCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.49411764705882355
Oligo Length: 170
This is Chapter 4
This is Segment 10
AGATGGCATGGACACGGAGATGACTAGCACTCAGCAGTGGACGCGTATCGTAAAGCAGCAGTAGCTAGCGCTCAGCGCTGCGTGCGCGGTGAGATAGCACACGTGCATAGTAGACAAGCACTAGCATAGTACGTGTCCACGAGCCATCTTCTCCAT
GC content of this strand: 

...

This is Chapter 35
This is Segment 10
AGTGCGAAGGAAGCCAACTAGGACTGCACGACGTACGCGGCGAGCAGCGTCGCGATCTATGATATGTGTGTAGTGACTAGTGTAGCACGTATAGTGTGACTAGAGTATGAGCACTAGTACACGTATAGCTTGCGACGTTGGACTCTTCT
GC content of this strand: 0.5033557046979866
Oligo Length: 149
This is Chapter 35
This is Segment 11
AGTGCGAAGGAAGCCAACTAGGACAGTGCTATGCGTCTGGACTCGCGTCACGCGTGCTCAGTCTATGGCGATGTGTACATAGTGTGACGCGACTAGAGTGATAGTCGTAGACGTCGGTCTGTGGTGCTATAGCACTAGCTAGTGCTCATCTTGCGACGTTGGACTCTTCT
GC content of this strand: 0.5294117647058824
Oligo Length: 170
This is Chapter 35
This is Segment 12
AGTGCGAAGGAAGCCAACTAGGAGTATGAGAAATACTCCGTGCGTAGACGAGCGTCGCATGTCATCATCGCGCAGAGTATGGTGTGACGCATAGTGTATCTGGTCTACAAGCAGATAGATCGTAGCACTAGTCTCAGAGTATCTTGCGACGTTGGACTCTTCT
GC content of this strand: 0.49079754601226994
Oligo Length: 163
This is Chapter 35
This is Segment 13
AGTGCGAAGGAAGCCAACTAGGAGAGACACGTGACAGATGTAGTACGACAGCAGCACGTAATAGTACGTATCTGGTACTCATCGCGCAGCTCGCAGTATCTGGCGATGTGTACAGTGATCGACAGCACTAGCTAGAGATAGCCTTGCGACGTTGGACTCTTCT
GC con

This is Chapter 36
This is Segment 23
AGAGGAGACCACCAGCTCTTAGCTGGTCGCTGGTCGCATGTCATCAGACGTCGGTACGCGGTGTAGTGAGATATGATCGACGAGACATGAGTCTAGTGTGCGGTCGTATATGTGACATGACAGATGAGCAGATGTATACGCTTGTGCCTCAACATGTCAACCT
GC content of this strand: 0.50920245398773
Oligo Length: 163
This is Chapter 36
This is Segment 24
AGAGGAGACCACCAGCTCTTAGCACGCTGTATGATAGATGCAGTGTGTCTGATGACTAGTGTCGCATGCGACTAGTATCATGAGACATAGCACTAAGCACTAAGCACTAGAGTGCGTCTACTGTGCCTCAACATGTCAACCT
GC content of this strand: 0.4788732394366197
Oligo Length: 142
This is Chapter 36
This is Segment 25
AGAGGAGACCACCAGCTCTTAGCAGGTCACTGGCGACGTGTCGAGTGCAGCACGTAGCGTGCTCTACGACGTCGGACTGTAGAGACATGTAGCTAGCTATATGCGACGTGTGCACGTCGACGCAGCACTAGTGATAGCGTCGTGTGCCTCAACATGTCAACCT
GC content of this strand: 0.558282208588957
Oligo Length: 163
This is Chapter 36
This is Segment 26
AGAGGAGACCACCAGCTCTTAGCATAGATGTGGTGTAGTTAGACGTGTAGACAGTACGCGGTGCGCGGTATCTGGACGTCGGTACGCGGTGTAGTGAGATATGATCGACGAGACATAGCACTAGACACAGTACGTTGTGCCTCAACATGTCAACCT
GC content of this strand: 0.51

This is Chapter 37
This is Segment 2
ACTTGGCATGTCCTACATCGTGTATGACGCACGACACGTGATCGACGACGATGGCAGATGGACGATGGATACGTGTGACTAGTATCTGGCTCACAGACGAGCGAGTGATAGTCACGGTGCACGGTCGCATGTCATCAGCTAGTGTGTCTGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.5294117647058824
Oligo Length: 170
This is Chapter 37
This is Segment 3
ACTTGGCATGTCCTACATCGTGTCGGTAGTCGGTCGCGCGCGCATAGACGAGCTCAGTATTCGCGCAGTCGCATGTCATCAGTCAGACGTGTACGGACTAGTGTCGCATGTATATATCTGACAGTAGTCGAGCACTAGACAGAGTATGAGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.5176470588235295
Oligo Length: 170
This is Chapter 37
This is Segment 4
ACTTGGCATGTCCTACATCGTGTCTGTATAGTGCAGATGGTACAGTTGACAGTAGCACACAGCAGTGGACTGTAGACTACGGACTGTAGTAGATGTGCTCTAATGAGCAGTGACTAGAGTATGGTATCTGGCTCTGGAGATGGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.4785276073619632
Oligo Length: 163
This is Chapter 37
This is Segment 5
ACTTGGCATGTCCTACATCGTGTCAGAGAGTAGTATCATGACGTGTTCTGTGTGCTACATGCTCTACGACATCGGCTCAGCGACTCGCGATCGTAGTATCTGGCGTCTGGTAGCGTGTCGCATGCACTCAATGAGCAGTCTGTGACGAGGCAAT

This is Chapter 37
This is Segment 47
ACTTGGCATGTCCTACATCGTTGTGGTAGACAGTGTCGCGCTCTGTGTATGACGAGCTGTAGCAGATGTCAGACGTCAGCGGTGCACGGCTATATTACGCTGGTATCATATGTCTGGTGCTATAGCACTAGCGTGCGAGATGGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.5153374233128835
Oligo Length: 163
This is Chapter 37
This is Segment 48
ACTTGGCATGTCCTACATCGTTGACGTCACGCGTACTATTATGCGCTAGACACGTCTCTAGTATCATGTACGACGTACATATCTATACGTGTCGCGCTCTGTGTACTATGACTCTGGCTGTCATCTATACAGCACTAGCACGAGTAGTCGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.4764705882352941
Oligo Length: 170
This is Chapter 37
This is Segment 49
ACTTGGCATGTCCTACATCGTTGAGGTAGTCGGTCGCGCGACTGTAGTCTGCAGACACACTAGCATAGACGTCGGTACGCGGTACTATGACGAGCGACGCTAGTGAGATATGAGCAGCTGTATGAGTATGGTATCTGGAGACGGCATCTGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.5235294117647059
Oligo Length: 170
This is Chapter 37
This is Segment 50
ACTTGGCATGTCCTACATCGTTGATGTAGTCGGTCGCGCGCTGTATGACGTCGGTAGATGGTATCATGTGTGACGCGACTAGACGTGTGACAGATGTGAGCAGTCTCGTGTGCGACGTACTATGTAGTGGTCTGCGCAATAGACGCTCTC

This is Chapter 39
This is Segment 15
ACCGTAGATCCTCGACAAGCTGATAGTCTGATGTCTGCAGTAGCTAGCTGCATGTATAGTGACGAGCGTCGTATGTGTGACGTCTGATGTCTGTGGTGACTAGTGAGATGTATCTGGTATCAGCATATGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.47651006711409394
Oligo Length: 149
This is Chapter 39
This is Segment 16
ACCGTAGATCCTCGACAAGCTGATCGCTAGTAGACGAGCGCATATAGACTAGTGTGATGTGACTGCGGTCTGATGTCTGCAGATCTATGACATCGGTCAGTAGAGTATGAGCACTAGTCGAGGCTACTGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.4966442953020134
Oligo Length: 149
This is Chapter 39
This is Segment 17
ACCGTAGATCCTCGACAAGCTGATGTACATGCGTGCGCGGTACAGTGTACATAGACTAGTGTCGCATGATCTATGTCGACGGTGCGTAGATATGTGTCACGCAGCACTAGTGATCGAGCAGGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5140845070422535
Oligo Length: 142
This is Chapter 39
This is Segment 18
ACCGTAGATCCTCGACAAGCTGCGTTATCAGCGCGTGCGGTCTCGTGACACACGCGCGACATGAGCAGTAGTCGGTCGCGCGTGCACGGAGTACAGACTAGTGACAGATTCTGCGTGTCGTCAGTCTCTAGTGACAGCGAGTGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5705521472392638
Ol